# Where should one open a bubble tea shop in Singapore?
### Applied Data Science Capstone by IBM/Coursera

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Results](#results)
* [Discussion](#discussion)
* [Conclusion](#conclusion)
* [References](#references)

## Introduction <a name="introduction"></a>

Bubble tea is a drink that is immensely popular in Southeast Asia, and has experienced a resurgence in popularity in recent years [1][2]. Although there was a lull in the expansion of the industry, it is now seeing a revival with new chains from China such as The Alley and Heytea entering the market. Globally, the bubble tea market is expected to reach 3.2 billion by 2023 [3], with Asia Pacific projected to be the second fastest growing region for the market. In this project, I will examine where a businessman, hoping to capitalize on this growing market, should open a bubble tea shop in Singapore. I will utilize foursquare API data to locate bubble tea shops in Singapore, and utilize demographic data to identify potential customers. Lastly, I will create a Bubble Tea Opportunity Index to show where the best subzones to open bubble tea shops are. Further detail can be found in the methodology section.



## Data <a name="data"></a>

While Singapore utilizes postal codes, there is a lack of granular public data (see Wikipedia [4]). However, the Singapore government splits the country up in 55 planning areas and 324 subzones from planning and statistical purposes. Hence, for the purpose of this project, I will be utilizing geographic data of the 324 subzones to represent neighbourhoods in Singapore. Geographic coordinates of each subzone are downloaded from a public Tableau map, where a csv file was generated, where I named it subzone.csv [5]. 

A kml file of the subzones was also downloaded from a government database [6]. This was then converted to a geojson file named sg_subzone.geojson. 

Foursquare API data is used to identify clusters and density of bubble tea shops in the various subzones in Singapore.

Demographic data (2018) for each subzone was downloaded from a public database from the Singapore government [7]. Since it was an xlsx file, I pre-cleaned it via excel to only include subzone name and the proportion of 20-39 year olds living in each subzone, as bubble tea is found to be most popular among this age group [8]. The csv file is named popdata.csv.

## Methodology <a name="methodology"></a>

First, I will utilize foursquare API data to locate bubble tea shops in different subzones in Singapore. I will then count the number of shops in each subzone, which indicates the level of competition in each subzone.

Following this, I will utilize demographic data to identify subzones with a higher proportion of potential customers. 


Additionally, I will also utilize demographic data and overlap the two, to examine if there are regions that have a higher proportion of youths (which tend to drink more bubble tea) but have a lower proportion of bubble tea shops, which would indicate an opportunity to open a bubble tea shop. According to a report on the bubble tea market [4], the target demographic are youths in their 20s to late 30s. Hence, I will be identifying the number of 20-39 in each subzone as an indicator of the amount of potential customers in each zone.

To complete the visualization, I will create a Bubble Tea Opportunity Index. This is done by first dividing the maximum number of bubble tea shops found in any subzone by the number of shops in an individual zone (so that the lower the number of shops, the higher the value as it indicates lower competition), and multiplying that value by the number of 20-39 found in each subzone. This value is then normalized to create an index from 0 to 1, where 0 indicates no opportunities while 1 indicates a huge opportunity to open bubble tea shops in that area.

## Results <a name="results"></a>

In this section, I will run through the code I used to create the results I found. 

### Cleaning Data
First, we import the packages required.


In [2]:
import geopandas as gpd
import pandas as pd
import numpy as np
import requests
import folium

Let's clean the subzone data.

In [3]:
df = pd.read_csv('subzone.csv', header=None, names=['Subzone','Geometry','Latitude','Longitude','Four Rm', 'Three Rm'])
df.drop(['Geometry','Four Rm','Three Rm'], axis=1, inplace=True)
df.drop(df.index[0], inplace=True)
df

,Subzone,Latitude,Longitude
1,YUNNAN,1.342866007,103.695040265
2,YUHUA WEST,1.348293786,103.729288239
3,YUHUA EAST,1.340444343,103.738925405
4,YISHUN WEST,1.433211485,103.830359267
5,YISHUN SOUTH,1.418969731,103.838429287
...,...,...,...
319,ALJUNIED,1.313854490,103.882436937
320,ALEXANDRA NORTH,1.292991557,103.820930976
321,ALEXANDRA HILL,1.287863531,103.810019163
322,AIRPORT ROAD,1.352110839,103.898214874


Let's clean the demographic data.

In [4]:
pop = pd.read_csv('popdata.csv')
pop2 = pop[pop.Subzone != 'Total']
pop3 = pop2.dropna()
pop4 = pop3.drop(['Total','20 - 24', '25 - 29', '30 - 34', '35 - 39'], axis =1)
pop5 = pop4.replace('-', 0)
pop5['Subzone'] = pop5['Subzone'].str.upper()
pop5['20-39'] = pd.to_numeric(pop5['20-39'])
pop5

,Subzone,20-39
1,ANG MO KIO TOWN CENTRE,1360.0
2,CHENG SAN,8380.0
3,CHONG BOON,7400.0
4,KEBUN BAHRU,6170.0
5,SEMBAWANG HILLS,1630.0
...,...,...
373,SPRINGLEAF,1080.0
374,YISHUN CENTRAL,470.0
375,YISHUN EAST,19880.0
376,YISHUN SOUTH,11860.0


Let's merge both dataframes for easier manipulation.

In [5]:
#merging df
data = pd.merge(df, pop5, on ='Subzone')
data1 = data.dropna()
data1['Latitude'] = pd.to_numeric(data1['Latitude'])
data1['Longitude'] = pd.to_numeric(data1['Longitude'])
data1

,Subzone,Latitude,Longitude,20-39
0,YUNNAN,1.342866,103.695040,21960.0
1,YUHUA WEST,1.348294,103.729288,5970.0
2,YUHUA EAST,1.340444,103.738925,7970.0
3,YISHUN WEST,1.433211,103.830359,16930.0
4,YISHUN SOUTH,1.418970,103.838429,11860.0
...,...,...,...,...
318,ALJUNIED,1.313854,103.882437,11420.0
319,ALEXANDRA NORTH,1.292992,103.820931,300.0
320,ALEXANDRA HILL,1.287864,103.810019,3920.0
321,AIRPORT ROAD,1.352111,103.898215,0.0


### Foursquare API data
Now, we have to obtain the number of bubble tea shops in each subzone. We do that through Foursquare.

First, we input our API credentials.

In [6]:
CLIENT_ID = 'CNYNIZU4MJ5MEOI0VTK4M3CGJ4L02QQXPCQR4IZEXLDSNA1B' # your Foursquare ID
CLIENT_SECRET = 'BUWO3BZBNW1ZI5FBF0QBPBVEPWBJ2RF2E0TWIUSBNB2M1CHD' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentials:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentials:
CLIENT_ID: CNYNIZU4MJ5MEOI0VTK4M3CGJ4L02QQXPCQR4IZEXLDSNA1B
CLIENT_SECRET:BUWO3BZBNW1ZI5FBF0QBPBVEPWBJ2RF2E0TWIUSBNB2M1CHD


Let's obtain the latitude and longitude values of each subzone from the dataframe we created earlier. We can test whether it works by printing out values of the first subzone.

In [7]:
subzone_latitude = data1.loc[0, 'Latitude'] # subzone latitude value
subzone_longitude = data1.loc[0, 'Longitude'] # subzone longitude value

subzone_name = data1.loc[0, 'Subzone'] # subzone name

print('Latitude and longitude values of {} are {}, {}.'.format(subzone_name, 
                                                               subzone_latitude, 
                                                               subzone_longitude))

Latitude and longitude values of YUNNAN are 1.3428660069999998, 103.695040265.


Let's create the url to query the API.

In [8]:
radius = 750 # define radius

LIMIT = 100

url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=52e81612bcbc57f1066b7a0c&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    subzone_latitude, 
    subzone_longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore?&categoryId=52e81612bcbc57f1066b7a0c&client_id=CNYNIZU4MJ5MEOI0VTK4M3CGJ4L02QQXPCQR4IZEXLDSNA1B&client_secret=BUWO3BZBNW1ZI5FBF0QBPBVEPWBJ2RF2E0TWIUSBNB2M1CHD&v=20180605&ll=1.3428660069999998,103.695040265&radius=750&limit=100'

Let's get the raw results for the first subzone.

In [9]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d63a0655d891b002c1f378c'},
 'response': {'headerLocation': 'Jurong West',
  'headerFullLocation': 'Jurong West, Singapore',
  'headerLocationGranularity': 'neighborhood',
  'query': 'bubble tea',
  'totalResults': 4,
  'suggestedBounds': {'ne': {'lat': 1.3496160137500066,
    'lng': 103.70177951961132},
   'sw': {'lat': 1.336116000249993, 'lng': 103.68830101038868}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5cc80ca933e118002cb88152',
       'name': 'Itea',
       'location': {'address': 'Jurong West',
        'crossStreet': 'Street 93',
        'lat': 1.341416,
        'lng': 103.691594,
        'labeledLatLngs': [{'label': 'display',
          'lat': 1.341416,
          'lng': 103.691594}],
        'distance': 416,
        'p

We then create a function that returns the category of the venues searched.

In [10]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

We then create the dataframe which shows the venue results of the first subzone.

In [15]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = pd.io.json.json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,Itea,Bubble Tea Shop,1.341416,103.691594
1,LiHO,Bubble Tea Shop,1.348438,103.697673
2,Itsbubblin,Bubble Tea Shop,1.348816,103.697590
3,Juz Bread @ Pioneer Mall,Bakery,1.341754,103.696443


As you can see, venues which are not categorized as bubble tea shops are also returned. This is because there are venues in Singapore that sell bubble tea as one of their secondary products. Since our project is focused on helping a businessman find opportunities to open a shop that solely sells bubble tea, shops that do not primarily sell bubble tea are not the main competition. Hence, we will filter them out. This will be done in the function created that will obtain bubble tea shops for each subzone. 

In [14]:
def getNearbyVenues(names, latitudes, longitudes, radius=750):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&categoryId=52e81612bcbc57f1066b7a0c&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            v['venue']['name'], 
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Subzone',  
                  'Venue',  
                  'Venue_Category']
    nearby_venues = nearby_venues[nearby_venues.Venue_Category == 'Bubble Tea Shop']
    
    return(nearby_venues)

Let's run the function to obtain the data.

In [17]:
sg_venues = getNearbyVenues(names=data1['Subzone'],
                                   latitudes=data1['Latitude'],
                                   longitudes=data1['Longitude']
                                  )

YUNNAN
YUHUA WEST
YUHUA EAST
YISHUN WEST
YISHUN SOUTH
YISHUN EAST
YISHUN CENTRAL
YIO CHU KANG WEST
YIO CHU KANG NORTH
YIO CHU KANG EAST
YIO CHU KANG
YEW TEE
XILIN
WOODLEIGH
WOODLANDS WEST
WOODLANDS SOUTH
WOODLANDS REGIONAL CENTRE
WOODLANDS EAST
WOODGROVE
WESTERN WATER CATCHMENT
WEST COAST
WENYA
WATERWAY EAST
VICTORIA
UPPER THOMSON
UPPER PAYA LEBAR
ULU PANDAN
TYERSALL
TURF CLUB
TUKANG
TUAS VIEW EXTENSION
TUAS VIEW
TUAS PROMENADE
TUAS NORTH
TUAS BAY
TRAFALGAR
TOWNSVILLE
TOH TUCK
TOH GUAN
TOA PAYOH WEST
TOA PAYOH CENTRAL
TIONG BAHRU STATION
TIONG BAHRU
THE WHARVES
TENGEH
TENGAH
TELOK BLANGAH WAY
TELOK BLANGAH RISE
TELOK BLANGAH DRIVE
TECK WHYE
TEBAN GARDENS
TANJONG RHU
TANJONG PAGAR
TANJONG IRAU
TANGLIN HALT
TANGLIN
TAMPINES WEST
TAMPINES NORTH
TAMPINES EAST
TAMAN JURONG
TAI SENG
TAGORE
SWISS CLUB
SUNSET WAY
SUNGEI ROAD
SUDONG
STRAITS VIEW
SPRINGLEAF
SOUTHERN GROUP
SOMERSET
SINGAPORE POLYTECHNIC
SINGAPORE GENERAL HOSPITAL
SIMPANG SOUTH
SIMPANG NORTH
SIMEI
SIGLAP
SHIPYARD
SHANGRI-LA
SERANG

Now that we have the data, we create a new column to count the number of shops in each subzone, and remove duplicates in the subzone column.

In [18]:
sg_venues['Bubble Tea Shop Count'] = sg_venues.groupby('Subzone')['Subzone'].transform('count')
sg_venues1 = sg_venues.drop_duplicates(subset='Subzone',keep='first')
sg_venues1

,Subzone,Venue,Venue_Category,Bubble Tea Shop Count
0,YUNNAN,Itea,Bubble Tea Shop,3
4,YUHUA WEST,Ding Tea 鼎茶,Bubble Tea Shop,2
8,YISHUN WEST,LiHO,Bubble Tea Shop,3
12,YISHUN SOUTH,Ulu,Bubble Tea Shop,1
17,YISHUN CENTRAL,KOI Cafè,Bubble Tea Shop,6
...,...,...,...,...
1055,ANSON,Woobbee,Bubble Tea Shop,6
1062,ANG MO KIO TOWN CENTRE,KOI Café,Bubble Tea Shop,5
1068,ALJUNIED,LiHO,Bubble Tea Shop,2
1071,ALEXANDRA NORTH,LiHO,Bubble Tea Shop,1


We then clean it, leaving just the subzone and count in the dataframe.

In [19]:
sg_venues2= sg_venues1.drop(['Venue','Venue_Category'], axis=1)
sg_venues2

,Subzone,Bubble Tea Shop Count
0,YUNNAN,3
4,YUHUA WEST,2
8,YISHUN WEST,3
12,YISHUN SOUTH,1
17,YISHUN CENTRAL,6
...,...,...
1055,ANSON,6
1062,ANG MO KIO TOWN CENTRE,5
1068,ALJUNIED,2
1071,ALEXANDRA NORTH,1


As you can see, there are subzones with no bubble tea shops (we started with 323 rows, but ended up with only 165 rows). Hence, we have to add the subzone that we left out back into the dataframe.

In [20]:
df2 = df.drop(['Latitude','Longitude'],axis=1)
df2['Bubble Tea Shop Count'] = 0
sg_venues3 = pd.concat([sg_venues2,df2])

sg_venues3.sort_values('Bubble Tea Shop Count', ascending=False)
sg_venues3.drop_duplicates(subset='Subzone', keep='first', inplace=True)
sg_venues3

,Subzone,Bubble Tea Shop Count
0,YUNNAN,3
4,YUHUA WEST,2
8,YISHUN WEST,3
12,YISHUN SOUTH,1
17,YISHUN CENTRAL,6
...,...,...
313,BANGKIT,0
317,ANCHORVALE,0
318,ANAK BUKIT,0
322,AIRPORT ROAD,0


### Creating the Index

We want to create an index which shows where it is the best to open a bubble tea shop. To do so, we first merge the dataframe we cleaned earlier with the dataframe with bubble tea shop counts. 

In [21]:
index_data = pd.merge(data1, sg_venues3, on ='Subzone')
index_data

,Subzone,Latitude,Longitude,20-39,Bubble Tea Shop Count
0,YUNNAN,1.342866,103.695040,21960.0,3
1,YUHUA WEST,1.348294,103.729288,5970.0,2
2,YUHUA EAST,1.340444,103.738925,7970.0,0
3,YISHUN WEST,1.433211,103.830359,16930.0,3
4,YISHUN SOUTH,1.418970,103.838429,11860.0,1
...,...,...,...,...,...
318,ALJUNIED,1.313854,103.882437,11420.0,2
319,ALEXANDRA NORTH,1.292992,103.820931,300.0,1
320,ALEXANDRA HILL,1.287864,103.810019,3920.0,4
321,AIRPORT ROAD,1.352111,103.898215,0.0,0


We then create a column with the percentage of 20-39 in each subzone.

We then create a column called opportunity to show the lower the number of bubble tea shops compared to the maximum number in any subzone, the higher the opportunity to open a shop there. To do so, we first add one to every subzone for Bubble Tea Shop Count, so that we avoid an infinity value later. We then divide the max number of shops by the shops in each subzone to create the value of opportunity. The higher the value, the lower the number of shops in the area and vice versa.

In [22]:
index_data['Bubble Tea Shop Count']= index_data['Bubble Tea Shop Count']+1
index_data['Opportunity']=index_data['Bubble Tea Shop Count'].max()/index_data['Bubble Tea Shop Count']
index_data

,Subzone,Latitude,Longitude,20-39,Bubble Tea Shop Count,Opportunity
0,YUNNAN,1.342866,103.695040,21960.0,4,7.5
1,YUHUA WEST,1.348294,103.729288,5970.0,3,10.0
2,YUHUA EAST,1.340444,103.738925,7970.0,1,30.0
3,YISHUN WEST,1.433211,103.830359,16930.0,4,7.5
4,YISHUN SOUTH,1.418970,103.838429,11860.0,2,15.0
...,...,...,...,...,...,...
318,ALJUNIED,1.313854,103.882437,11420.0,3,10.0
319,ALEXANDRA NORTH,1.292992,103.820931,300.0,2,15.0
320,ALEXANDRA HILL,1.287864,103.810019,3920.0,5,6.0
321,AIRPORT ROAD,1.352111,103.898215,0.0,1,30.0


We then create the index by multiplying the newly made columns.

In [23]:
index_data['Bubble Tea Opportunity Index Raw']=index_data['20-39']*index_data['Opportunity']
index_data

,Subzone,Latitude,Longitude,20-39,Bubble Tea Shop Count,Opportunity,Bubble Tea Opportunity Index Raw
0,YUNNAN,1.342866,103.695040,21960.0,4,7.5,164700.0
1,YUHUA WEST,1.348294,103.729288,5970.0,3,10.0,59700.0
2,YUHUA EAST,1.340444,103.738925,7970.0,1,30.0,239100.0
3,YISHUN WEST,1.433211,103.830359,16930.0,4,7.5,126975.0
4,YISHUN SOUTH,1.418970,103.838429,11860.0,2,15.0,177900.0
...,...,...,...,...,...,...,...
318,ALJUNIED,1.313854,103.882437,11420.0,3,10.0,114200.0
319,ALEXANDRA NORTH,1.292992,103.820931,300.0,2,15.0,4500.0
320,ALEXANDRA HILL,1.287864,103.810019,3920.0,5,6.0,23520.0
321,AIRPORT ROAD,1.352111,103.898215,0.0,1,30.0,0.0


Using scikit-learn, we then normalize the index.

In [24]:
from sklearn import preprocessing
index_data1 = index_data[['Bubble Tea Opportunity Index Raw']].values.astype(float)
min_max_scaler = preprocessing.MinMaxScaler()
# Create an object to transform the data to fit minmax processor
index_data1_scaled = min_max_scaler.fit_transform(index_data1)

# Run the normalizer on the dataframe
index_data_norm = pd.DataFrame(index_data1_scaled)
index_data_norm

,0
0,0.234615
1,0.085043
2,0.340598
3,0.180876
4,0.253419
...,...
318,0.162678
319,0.006410
320,0.033504
321,0.000000


Now, we added the normalized index as a column.

In [25]:
index_data2 = pd.merge(index_data, index_data_norm, left_index=True, right_index=True)
index_data2.rename(columns={0:'Bubble Tea Opportunity Index'}, inplace=True)
index_data2.dropna(inplace=True)
index_data2.reset_index(inplace=True)
index_data2.drop(['index'], axis=1, inplace=True)
index_data2

,Subzone,Latitude,Longitude,20-39,Bubble Tea Shop Count,Opportunity,Bubble Tea Opportunity Index Raw,Bubble Tea Opportunity Index
0,YUNNAN,1.342866,103.695040,21960.0,4,7.5,164700.0,0.234615
1,YUHUA WEST,1.348294,103.729288,5970.0,3,10.0,59700.0,0.085043
2,YUHUA EAST,1.340444,103.738925,7970.0,1,30.0,239100.0,0.340598
3,YISHUN WEST,1.433211,103.830359,16930.0,4,7.5,126975.0,0.180876
4,YISHUN SOUTH,1.418970,103.838429,11860.0,2,15.0,177900.0,0.253419
...,...,...,...,...,...,...,...,...
318,ALJUNIED,1.313854,103.882437,11420.0,3,10.0,114200.0,0.162678
319,ALEXANDRA NORTH,1.292992,103.820931,300.0,2,15.0,4500.0,0.006410
320,ALEXANDRA HILL,1.287864,103.810019,3920.0,5,6.0,23520.0,0.033504
321,AIRPORT ROAD,1.352111,103.898215,0.0,1,30.0,0.0,0.000000


We then remove unused columns in both index_data2.

In [26]:
index_data3=index_data2.drop(['Latitude','Longitude','Bubble Tea Opportunity Index Raw','Opportunity'], axis=1)
index_data3['Bubble Tea Shop Count']= index_data3['Bubble Tea Shop Count']-1
index_data3

,Subzone,20-39,Bubble Tea Shop Count,Bubble Tea Opportunity Index
0,YUNNAN,21960.0,3,0.234615
1,YUHUA WEST,5970.0,2,0.085043
2,YUHUA EAST,7970.0,0,0.340598
3,YISHUN WEST,16930.0,3,0.180876
4,YISHUN SOUTH,11860.0,1,0.253419
...,...,...,...,...
318,ALJUNIED,11420.0,2,0.162678
319,ALEXANDRA NORTH,300.0,1,0.006410
320,ALEXANDRA HILL,3920.0,4,0.033504
321,AIRPORT ROAD,0.0,0,0.000000


### Map of Index

Finally, let's create the map. We begin by reading the geojson file downloaded from the public database using geopandas.

In [27]:
gdf = gpd.read_file("https://raw.githubusercontent.com/brendantwj/Capstone/master/sg_subzone.geojson")
gdf

,Name,description,X_ADDR,SHAPE_Length,FMEL_UPD_D,Region Name,Planning Area Name,Region Code,Subzone Name,Central Area Indicator,SHAPE,Y_ADDR,Subzone Code,INC_CRC,Subzone Number,SHAPE_Area,Planning Area Code,Field_1,geometry
0,BUKIT MERAH,None,26360.799,3074.963235,5/11/2016 4:06:21 PM,CENTRAL REGION,BUKIT MERAH,CR,BUKIT MERAH,N,Polygon,29384.1429,BMSZ02,1C51019439A68700,2,411722.822583,BM,BUKIT MERAH,(POLYGON ((103.8220876771419 1.280494489593267...
1,CHINA SQUARE,None,29509.6437,1605.279697,5/11/2016 4:06:21 PM,CENTRAL REGION,OUTRAM,CR,CHINA SQUARE,Y,Polygon,29646.4495,OTSZ04,B609DF5587626C8F,4,133006.936012,OT,CHINA SQUARE,(POLYGON ((103.8485887402583 1.282316766729342...
2,PHILLIP,None,29706.7242,871.554888,5/11/2016 4:06:21 PM,CENTRAL REGION,DOWNTOWN CORE,CR,PHILLIP,Y,Polygon,29744.9079,DTSZ04,615D4EDDEF809F8E,4,39437.93527,DT,PHILLIP,(POLYGON ((103.8496190494575 1.284119120671727...
3,RAFFLES PLACE,None,29968.6175,1872.752161,5/11/2016 4:06:21 PM,CENTRAL REGION,DOWNTOWN CORE,CR,RAFFLES PLACE,Y,Polygon,29572.7618,DTSZ05,72107B11807074F4,5,188767.489706,DT,RAFFLES PLACE,(POLYGON ((103.8525272809205 1.286165686174613...
4,BAYFRONT SUBZONE,None,30806.2418,2897.126426,5/11/2016 4:06:21 PM,CENTRAL REGION,DOWNTOWN CORE,CR,BAYFRONT SUBZONE,Y,Polygon,29530.167,DTSZ12,197F5E664DA4D5E1,12,521200.522376,DT,BAYFRONT SUBZONE,(POLYGON ((103.8588808156329 1.278738667748894...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,SENOKO NORTH,None,24665.7878,7392.128851,5/11/2016 4:06:21 PM,NORTH REGION,SEMBAWANG,NR,SENOKO NORTH,N,Polygon,49482.5953,SBSZ08,A800CBEE879C1BF9,8,2241386.61934,SB,SENOKO NORTH,(POLYGON ((103.8157697436791 1.470615367672462...
319,CHANGI POINT,None,44603.6326,4897.482363,5/11/2016 4:06:21 PM,EAST REGION,CHANGI,ER,CHANGI POINT,N,Polygon,41360.4186,CHSZ01,0F530A1557E9AF1F,1,995944.765436,CH,CHANGI POINT,(POLYGON ((103.9823594617553 1.393557231580208...
320,SENGKANG WEST,None,31319.9431,5991.020008,5/11/2016 4:06:21 PM,NORTH-EAST REGION,SENGKANG,NER,SENGKANG WEST,N,Polygon,41936.6786,SESZ07,53C82220D68B4128,7,1670029.855661,SE,SENGKANG WEST,"(POLYGON ((103.868920291105 1.401031940545582,..."
321,PASIR RIS WAFER FAB PARK,None,38612.4965,10606.646904,5/11/2016 4:06:21 PM,EAST REGION,PASIR RIS,ER,PASIR RIS WAFER FAB PARK,N,Polygon,41196.6819,PRSZ01,9658EA18C574343B,1,5325421.575164,PR,PASIR RIS WAFER FAB PARK,(POLYGON ((103.9347791179268 1.400988154975988...


Let's add the index to the geojson file.

In [28]:
gdf1=gdf.merge(index_data3, left_on='Name', right_on='Subzone', how='left')
gdf1

,Name,description,X_ADDR,SHAPE_Length,FMEL_UPD_D,Region Name,Planning Area Name,Region Code,Subzone Name,Central Area Indicator,...,INC_CRC,Subzone Number,SHAPE_Area,Planning Area Code,Field_1,geometry,Subzone,20-39,Bubble Tea Shop Count,Bubble Tea Opportunity Index
0,BUKIT MERAH,None,26360.799,3074.963235,5/11/2016 4:06:21 PM,CENTRAL REGION,BUKIT MERAH,CR,BUKIT MERAH,N,...,1C51019439A68700,2,411722.822583,BM,BUKIT MERAH,(POLYGON ((103.8220876771419 1.280494489593267...,BUKIT MERAH,290.0,1,0.006197
1,CHINA SQUARE,None,29509.6437,1605.279697,5/11/2016 4:06:21 PM,CENTRAL REGION,OUTRAM,CR,CHINA SQUARE,Y,...,B609DF5587626C8F,4,133006.936012,OT,CHINA SQUARE,(POLYGON ((103.8485887402583 1.282316766729342...,CHINA SQUARE,370.0,15,0.000988
2,PHILLIP,None,29706.7242,871.554888,5/11/2016 4:06:21 PM,CENTRAL REGION,DOWNTOWN CORE,CR,PHILLIP,Y,...,615D4EDDEF809F8E,4,39437.93527,DT,PHILLIP,(POLYGON ((103.8496190494575 1.284119120671727...,PHILLIP,0.0,16,0.000000
3,RAFFLES PLACE,None,29968.6175,1872.752161,5/11/2016 4:06:21 PM,CENTRAL REGION,DOWNTOWN CORE,CR,RAFFLES PLACE,Y,...,72107B11807074F4,5,188767.489706,DT,RAFFLES PLACE,(POLYGON ((103.8525272809205 1.286165686174613...,RAFFLES PLACE,0.0,14,0.000000
4,BAYFRONT SUBZONE,None,30806.2418,2897.126426,5/11/2016 4:06:21 PM,CENTRAL REGION,DOWNTOWN CORE,CR,BAYFRONT SUBZONE,Y,...,197F5E664DA4D5E1,12,521200.522376,DT,BAYFRONT SUBZONE,(POLYGON ((103.8588808156329 1.278738667748894...,BAYFRONT SUBZONE,0.0,2,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
318,SENOKO NORTH,None,24665.7878,7392.128851,5/11/2016 4:06:21 PM,NORTH REGION,SEMBAWANG,NR,SENOKO NORTH,N,...,A800CBEE879C1BF9,8,2241386.61934,SB,SENOKO NORTH,(POLYGON ((103.8157697436791 1.470615367672462...,SENOKO NORTH,0.0,0,0.000000
319,CHANGI POINT,None,44603.6326,4897.482363,5/11/2016 4:06:21 PM,EAST REGION,CHANGI,ER,CHANGI POINT,N,...,0F530A1557E9AF1F,1,995944.765436,CH,CHANGI POINT,(POLYGON ((103.9823594617553 1.393557231580208...,CHANGI POINT,160.0,0,0.006838
320,SENGKANG WEST,None,31319.9431,5991.020008,5/11/2016 4:06:21 PM,NORTH-EAST REGION,SENGKANG,NER,SENGKANG WEST,N,...,53C82220D68B4128,7,1670029.855661,SE,SENGKANG WEST,"(POLYGON ((103.868920291105 1.401031940545582,...",SENGKANG WEST,0.0,0,0.000000
321,PASIR RIS WAFER FAB PARK,None,38612.4965,10606.646904,5/11/2016 4:06:21 PM,EAST REGION,PASIR RIS,ER,PASIR RIS WAFER FAB PARK,N,...,9658EA18C574343B,1,5325421.575164,PR,PASIR RIS WAFER FAB PARK,(POLYGON ((103.9347791179268 1.400988154975988...,PASIR RIS WAFER FAB PARK,0.0,0,0.000000


Now, we create the legend for the map.

In [29]:
variable = 'Bubble Tea Opportunity Index'
gdf1=gdf1.sort_values(by=variable, ascending=True)
colormap = folium.LinearColormap(colors=["red","orange","yellow","green"],vmin=0, vmax=1).to_step(n=5)
colormap.caption = "Bubble Tea Opportunity Index"
colormap

Finally, we create the choropleth map for the index.

In [33]:


centroid=gdf1.geometry.centroid

m=folium.Map(location=[centroid.y.mean(), centroid.x.mean()], zoom_start=11, tiles='stamentoner')

folium.GeoJson(gdf1[['geometry','Name',variable,'20-39','Bubble Tea Shop Count']],
               name="Singapore",
               style_function=lambda x: {"weight":1, 'color':'black','fillColor':colormap(x['properties'][variable]), 'fillOpacity':0.7},
              highlight_function=lambda x: {'weight':3, 'color':'black'},
               smooth_factor=1.0,
              tooltip=folium.features.GeoJsonTooltip(fields=['Name',variable,'20-39','Bubble Tea Shop Count'],
                                              aliases=['Subzone','Bubble Tea Opportunity Index','Number of 20-39','Bubble Tea Shop Count'], 
                                              labels=True,
                                              localize=False,
                                              sticky=True,
                                              style=None
                                             )
              ).add_to(m)
colormap.add_to(m)

m

## Discussion <a name="discussion"></a>

Our analysis shows that there are several areas where the opportunity to open bubble tea shops still exist in Singapore. This can be seen by the color coding of each subzone according to the index. Additional information could also be easily seen via the tooltips.

These subzones that have the highest index values include the Bedok North, Tampines East and Yishun East. Other than these prime areas, other subzones with lower index values were also found, such as Pasir Ris Drive. In essence, the map would provide a quick overview for any businessman looking to enter the bubble tea industry in Singapore. 

There are certain limitations to our analysis. For one, only residential data was used. Given the small size of Singapore, where people work is also highly relevant. However, such data was not included as I was unable to find it in any public database. This can be added in future iterations of the index.

Additionally, we can also add other factors that might affect potential business to the index. For instance, it is quite likely that places with a higher proportion of entertainment venues such as arcades may be places of higher potential, since people are more likely to spend time around the area and may thus buy more bubble tea.

## Conclusion <a name="conclusion"></a>

The purpose of this project was to identify areas in Singapore that is best for a new bubble tea shop. This was accomplished via a choropleth map showing a Bubble Tea Opportunity Index. By gathering data from Foursquare API as well as various public databases, we were able to create an index that showed the best areas in Singapore to open a bubble tea shop. Further improvements can be made by adding more variables to the index, so that it is more comprehensive. 

## References <a name="references"></a>

1.	https://www.mintel.com/blog/foodservice-market-news/milk-tea-craze-bubbles-up-in-southeast-asia
2.	https://www.straitstimes.com/videos/consuming-singapore-the-obsession-with-bubble-tea
3.	https://www.alliedmarketresearch.com/press-release/bubble-tea-market.html
4.	https://en.wikipedia.org/wiki/Postal_codes_in_Singapore
5.	https://public.tableau.com/profile/solomon.teo#!/vizhome/FacebookCheckin/SubZone
6.  https://data.gov.sg/dataset/master-plan-2014-subzone-boundary-web
7.  https://data.gov.sg/dataset/resident-population-by-planning-area-subzone-age-group-and-sex-2015
8.  https://static1.squarespace.com/static/56e85bd540261db9be8c125b/t/56eba2a827d4bdff1f7e3083/1458283178494/Target+Market+Project+-+Final+-+Jon+Field.pdf

##### Brendan Tan (2019)